In [1]:
import pandas as pd
from tokenizers import Tokenizer
from transformers import AutoTokenizer

In [2]:
train_law_par = pd.read_parquet(r"c:\Users\siren\Desktop\1507\dataset\hukuk_raw_dataset\train.parquet", engine="pyarrow")

In [3]:
old_tokenizer = AutoTokenizer.from_pretrained("redrussianarmy/gpt2-turkish-cased")

c:\Users\siren\anaconda3\envs\tubitak\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
text_data = train_law_par["text"].tolist()
def law_iter():
    for text in text_data:
        yield text

In [5]:
tokenizer =old_tokenizer.train_new_from_iterator(law_iter(), 52000)

In [6]:
sen_enc3 = tokenizer.encode(train_law_par["text"][1010])
print(sen_enc3)
for token in sen_enc3:
    print(token, tokenizer.decode(token))

[3536, 476, 641, 539, 984, 1038, 686, 8544, 652, 5392, 612, 308, 24735, 9891, 3756, 2543, 11585, 12016, 14, 1800, 49682, 12, 5676, 294, 3221, 18113, 31201, 649, 17914, 2327, 2042, 10110, 6779, 1198, 5822, 3332, 1978, 15816, 687, 15038, 14, 2703, 841, 8088, 48080, 304, 1693, 405, 12, 21040, 311, 8477, 6585, 304, 9723, 6221, 14, 18854, 3226, 5097, 1082, 304, 8720, 1469, 14573, 912, 5921, 308, 1050, 1507, 21218, 984, 10684, 27490, 12493, 14, 2703, 21423, 3715, 2343, 649, 4148, 5798, 841, 8088, 33764, 3687, 1162, 1952, 14, 2862, 330, 14, 19, 14, 18, 14, 14880, 320, 912, 356, 402, 25830, 480, 9, 47503, 461, 2024, 5630, 18119, 5536, 12, 15316, 2899, 11763, 641, 3399, 4123, 304, 760, 2389, 294, 3596, 8337, 21693, 6123, 7836, 5818, 885, 8878, 728, 324, 8513, 18383, 1263, 21754, 47953, 16082, 628, 2737, 14, 3457, 1412, 304, 1693, 405, 841, 8088, 8232, 7634, 294, 3637, 40262, 458, 6210, 841, 9381, 6394, 304, 4528, 3215, 5587, 14, 2862, 2121, 8088, 25180, 458, 29790, 4991, 840, 840, 841, 7539, 11

In [9]:
tokenizer.eos_token

'<|endoftext|>'

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
tokenizer.push_to_hub("sergeantson/1507_Law_Tokenizer")

CommitInfo(commit_url='https://huggingface.co/sergeantson/1507_Law_Tokenizer/commit/7a187ea85f2caa10d9a31a3840de9082c8f48bd0', commit_message='Upload tokenizer', commit_description='', oid='7a187ea85f2caa10d9a31a3840de9082c8f48bd0', pr_url=None, pr_revision=None, pr_num=None)